# [Keras + Universal Sentence Encoder = Deep Meter] (https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/) 

This notebook creates an autoencoder using the Universal Sentence Encoder. The autoencoder output is CMUdict syllables. The dataset is that subset of Allison Parrish's Project Gutenberg poetry archive which happens to scan in iambic pentameter.

The notebook is based on Chengwei Zhang's example of wrapping the USE inside a larger tensorflow model saves to a Keras model (without save the USE itself in the TF model).

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.

Since there are 10 one-hot values for 10 sets of 6k syllables, this is "multi-label classification"
Changes for multi-label classification:
sigmoid activation instead of softmax
binary_crossentropy

Text format is tab-separated, 2 columns: first text, second multi-level
array of syllables:

Multi-output version

# Getting Started

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [1]:
# Install the latest Tensorflow version.
#!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
#!pip3 install --quiet tensorflow-hub
#%cd /content
!git clone https://github.com/LanceNorskog/deep_meter || true
%cd /content/deep_meter
!git pull
# could not figure out how to read gzipped files as text!
!gunzip -qf blobs/*.gz || true
!gunzip -qf prepped_data/*.gz || true

fatal: destination path 'deep_meter' already exists and is not an empty directory.
/content/deep_meter
Already up to date.
gzip: blobs/*.gz: No such file or directory
gzip: prepped_data/*.gz: No such file or directory


In [2]:
# boilerplate from base notebook
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Nadam, Adam
np.random.seed(10)

Using TensorFlow backend.


In [0]:
# github deep_meter code
import utils
# should not need this to use utils.flatten but is true anyway?
from itertools import chain
import subprocess
import syllables
# misc for this notebook
from ast import literal_eval

import scipy



In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
#embed = hub.Module(module_url)
#embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

In [0]:
# read classified poetry lines: text tab [['syll', 'la', 'ble'], ...]
# clip to only most common syllables with syllable manager
# ['words', ...], [[[0,0,1,0], ...]]
def get_data(filename, syll_mgr, num_symbols):
    num_syllables = syll_mgr.get_size()      
    lines = open(filename, 'r').read().splitlines()
    num_lines = len(lines)
    #num_lines = 4200
    text_lines = []
    text_sylls = []
    for i in range(0, num_lines):
      parts = lines[i].split("\t")
      label = utils.flatten(literal_eval(parts[1]))
      if len(label) == num_symbols:
        text_lines.append(str(parts[0]))
        text_sylls.append(label)
    num_lines = len(text_lines)
    label_array = np.zeros((num_symbols, num_lines, num_syllables), dtype=np.int8)
    for i in range(0, num_lines):
      for j in range(num_symbols):
        label_array[j][i][syll_mgr.get_encoding(text_sylls[i][j])] = 1

    return (text_lines, label_array)


In [0]:
# syllables in descending order of occurrence - 6k in gutenberg.iambic_pentameter, 15k total
# clamp to most common 100 syllables while debugging- use NCE to get all syllables or interesting number
# 98 + pause + wildcard
# iambic pentameter
num_symbols = 10
#syll_mgr = syllables.syllables(num_syllables)
syll_mgr = syllables.syllables()
num_syllables = syll_mgr.get_size() 



In [24]:
(train_text, train_label) = get_data('prepped_data/gutenberg.iambic_pentameter.train', syll_mgr, num_symbols)
num_training = len(train_text)
#train_text = train_text[0:100]
#train_label = train_label[0:100]

(test_text, test_label) = get_data('prepped_data/gutenberg.iambic_pentameter.test', syll_mgr, num_symbols)
#test_text = test_text[0:100]
#test_label = test_label[0:100]

num_testing = len(test_text)
print(len(train_text))
print(len(test_text))
print(train_label.shape)
print(test_label.shape)

3975
3943
(10, 3975, 6635)
(10, 3943, 6635)


## Embed training & test text

In [25]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)
# important?
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)
print(type(train_text))
#train_text_t = tf.convert_to_tensor(train_text, dtype='string', name='training_text')
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  train_embeddings = session.run(embed(train_text))
  test_embeddings = session.run(embed(test_text))
train_text_d = np.array(train_embeddings)
test_text_d = np.array(test_embeddings)
print(train_text_d.shape)
print(test_text_d.shape)
# conserve space
embed = None
train_text = None
train_embeddings = None

<class 'list'>
[ 0.05363204  0.07069162 -0.00749868]
(3975, 512)
(3943, 512)


In [26]:
print(test_text_d.shape)
# slow
num_epochs = 40
adam_0001 = Adam(0.0001)
adam_0001 = tf.contrib.opt.NadamOptimizer(0.0001)

(3943, 512)


## Assemble model

In [27]:
input_embeddings = layers.Input(shape=(512,), dtype=tf.float32, name='Input')
dropout_input = layers.Dropout(0.2)(input_embeddings)
dense = layers.Dense(768, activation='relu', name='Convoluted')(dropout_input)
pred_array = []
loss_array = []
names_array = []
for i in range(num_symbols):
  name = 'Flatout'+str(i)
  pred_array.append(layers.Dense(num_syllables, activation='softmax', name=name)(dense))
  loss_array.append('categorical_crossentropy')
  names_array.append(name)
model = Model(inputs=input_embeddings, outputs=pred_array)
model.compile(loss=loss_array, 
              optimizer=adam_0001, 
              metrics=['categorical_accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 512)          0                                            
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 512)          0           Input[0][0]                      
__________________________________________________________________________________________________
Convoluted (Dense)              (None, 768)          393984      dropout_2[0][0]                  
__________________________________________________________________________________________________
Flatout0 (Dense)                (None, 6635)         5102315     Convoluted[0][0]                 
__________________________________________________________________________________________________
Flatout1 (

## Train Keras model and save weights
This only trains and save our Keras layers not the embed module' weights.

In [28]:
use_saved_model=False

print(train_label.shape)
if not use_saved_model or not os.path.exists('./model.h5'):
  with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    train_labels = []
    test_labels = []
    for i in range(num_symbols):
        train_labels.append(np.array(train_label[i]))
        test_labels.append(np.array(test_label[i]))
    history = model.fit(train_text_d, 
            train_labels,
            #validation_data=(test_text_d, test_labels),
            epochs=num_epochs,
            callbacks = [EarlyStopping(patience=2)],
            batch_size=32,
            verbose=2)
    model.save_weights('./model.h5')


(10, 3975, 6635)
Epoch 1/2
 - 19s - loss: 83.9290 - Flatout0_loss: 8.0308 - Flatout1_loss: 8.5786 - Flatout2_loss: 8.2678 - Flatout3_loss: 8.5763 - Flatout4_loss: 8.2961 - Flatout5_loss: 8.5288 - Flatout6_loss: 8.2544 - Flatout7_loss: 8.5430 - Flatout8_loss: 8.2648 - Flatout9_loss: 8.5885 - Flatout0_categorical_accuracy: 0.2226 - Flatout1_categorical_accuracy: 0.0091 - Flatout2_categorical_accuracy: 0.1165 - Flatout3_categorical_accuracy: 0.0103 - Flatout4_categorical_accuracy: 0.0546 - Flatout5_categorical_accuracy: 0.0259 - Flatout6_categorical_accuracy: 0.1190 - Flatout7_categorical_accuracy: 0.0257 - Flatout8_categorical_accuracy: 0.1001 - Flatout9_categorical_accuracy: 0.0050
Epoch 2/2


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,Flatout0_loss,Flatout1_loss,Flatout2_loss,Flatout3_loss,Flatout4_loss,Flatout5_loss,Flatout6_loss,Flatout7_loss,Flatout8_loss,Flatout9_loss,Flatout0_categorical_accuracy,Flatout1_categorical_accuracy,Flatout2_categorical_accuracy,Flatout3_categorical_accuracy,Flatout4_categorical_accuracy,Flatout5_categorical_accuracy,Flatout6_categorical_accuracy,Flatout7_categorical_accuracy,Flatout8_categorical_accuracy,Flatout9_categorical_accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 9s - loss: 60.6846 - Flatout0_loss: 3.7953 - Flatout1_loss: 7.1661 - Flatout2_loss: 5.4188 - Flatout3_loss: 7.1444 - Flatout4_loss: 5.5014 - Flatout5_loss: 6.8688 - Flatout6_loss: 5.2603 - Flatout7_loss: 6.9558 - Flatout8_loss: 5.3482 - Flatout9_loss: 7.2254 - Flatout0_categorical_accuracy: 0.2136 - Flatout1_categorical_accuracy: 0.0171 - Flatout2_categorical_accuracy: 0.1374 - Flatout3_categorical_accuracy: 0.0138 - Flatout4_categorical_accuracy: 0.1192 - Flatout5_categorical_accuracy: 0.0357 - Flatout6_categorical_accuracy: 0.1560 - Flatout7_categorical_accuracy: 0.0317 - Flatout8_categorical_accuracy: 0.1228 - Flatout9_categorical_accuracy: 0.0078


In [29]:
!ls -alh | grep model.h5

-rw-r--r-- 1 root root 197M Oct 31 16:40 model.h5


## Make predictions

In [31]:

#new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(test_text_d, batch_size=32)

print(len(predicts))
print(predicts[0].shape)

10
(3943, 6635)


In [32]:
first = predicts[0]
max_d = -1000000
min_d = 1000000
max_i = 0
min_i = 0
for i in range(len(predicts) - 1):
  v = predicts[i]
  d = scipy.spatial.distance.cosine(v, first)
  if max_d < d:
    max_d = d
    max_i = i
  if min_d > d:
    min_d = d
    min_i = i
print("Min and max distances: {0} and {1}".format(min_d, max_d))
print(test_text[max_i])
syll_mgr.interpret2(predicts[max_i])

ValueError: ignored

In [34]:
for i in range(10,100):
  print(test_text[i])
  #syll_mgr.interpret2(predicts[i])
  pred = []
  for j in range(num_symbols):
    pred.append(predicts[j][i])
  syll_mgr.interpret2(utils.flatten(pred))
  print('')

Of humble seeming: and, behind them all,
[3, 20, 2, 5, 2, 5, 2, 5, 2, 76]
['AE N D', 'AO L', 'DH AH', 'AH V', 'DH AH', 'AH V', 'DH AH', 'AH V', 'DH AH', 'D EY']

Behind his head upon the shoulders lay,
[3, 20, 2, 5, 2, 3, 2, 5, 2, 107]
['AE N D', 'AO L', 'DH AH', 'AH V', 'DH AH', 'AE N D', 'DH AH', 'AH V', 'DH AH', 'N AY T']

Discover countries, with a kindred heart
[3, 20, 2, 5, 2, 5, 2, 5, 2, 76]
['AE N D', 'AO L', 'DH AH', 'AH V', 'DH AH', 'AH V', 'DH AH', 'AH V', 'DH AH', 'D EY']

Was open in his praise, and plaudits rose
[3, 20, 2, 5, 2, 3, 2, 5, 2, 76]
['AE N D', 'AO L', 'DH AH', 'AH V', 'DH AH', 'AE N D', 'DH AH', 'AH V', 'DH AH', 'D EY']

A spear the hero bore of wondrous strength,
[3, 20, 2, 5, 2, 3, 2, 5, 2, 76]
['AE N D', 'AO L', 'DH AH', 'AH V', 'DH AH', 'AE N D', 'DH AH', 'AH V', 'DH AH', 'D EY']

Were things indifferent to the Shepherds thoughts.
[3, 20, 2, 5, 2, 5, 2, 5, 2, 62]
['AE N D', 'AO L', 'DH AH', 'AH V', 'DH AH', 'AH V', 'DH AH', 'AH V', 'DH AH', 'W EY']

That t

In [0]:
categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
print("Categorie: {0}".format(categories))
predict_labels = [categories[logit] for logit in predict_logits]
predict_labels

In [0]:

os.remove('./model.h5')